In [ ]:
import nltk
import pandas as pd
import re
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('brown')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
from gensim.models import KeyedVectors
filename = 'path to word vectors'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Noun phrase extractor
from textblob import TextBlob
def noun_phrases(text):
  np_text = []
  sentences = nltk.sent_tokenize(text)
  for sentence in sentences:
    blob = TextBlob(sentence)
    np_sent = blob.noun_phrases
    np_text.append(np_sent)
  return np_text

In [ ]:
# Topic modeller
def topic_model(text):
  import nltk
  import spacy
  from spacy.lang.en import English
  parser = English() # parser is imported
  def tokenize(text):
    lda_tokens = []
    tokens = parser(text) # text is converted to tokens
    for token in tokens: # lda tokens are created for each word
      if token.orth_.isspace():
        continue
      elif token.like_url:
        lda_tokens.append('URL')
      elif token.orth_.startswith('@'):
        lda_tokens.append('SCREEN_NAME')
      else:
        lda_tokens.append(token.lower_)
    return lda_tokens
  from nltk.corpus import wordnet as wn
  def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
      return word
    else:
      return lemma
  from nltk.stem.wordnet import WordNetLemmatizer
  def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
  en_stop = set(nltk.corpus.stopwords.words('english'))
  
  def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens
  
  from gensim import corpora
  import gensim
  import pickle
  import random
  text_data = []
  data = text
  data = nltk.sent_tokenize(data)
  for i in data :
    for line in i.split("."):
      tokens = prepare_text_for_lda(line)
      text_data.append(tokens)
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text)for text in text_data]
    #corpus
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    NUM_TOPICS = 1
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=50)
    ldamodel.save('model5.gensim')
    topics = ldamodel.print_topics(num_words=30)
    t = []
    for topic in topics:
      t.append(topic)
    import re
    t = re.sub("[^a-zA-Z]",".",str(t))
    t = re.sub("[.]+"," ",t)
    t = nltk.word_tokenize(t)
    return t

In [ ]:
def find_distance(text1,text2):
  try:
    value = model.distance(text1,text2)
  except KeyError:
    value = 1
  return 1-value

Statistical Scores

In [ ]:
import math
def tf_isf_score(text):
  sentences = sent_tokenize(text)
  sent_count = len(sentences)
  tf = dict()
  sf = dict()
  words_set = []
  scores = []
  for sentence in sentences:
    words = word_tokenize(sentence)
    for word in words:
      words_set.append(word)
      if (word,sentence) in tf:
        tf[word,sentence]+=1
      else:
        tf[word,sentence]=1
  words_set = set(words_set)
  for word in words_set:
    sf[word]=0
    for sentence in sentences:
      words = word_tokenize(sentence)
      for word2 in words:
        if word2 == word:
          sf[word]+=1
          continue
  for sentence in sentences:
    words = word_tokenize(sentence)
    score = 0
    for word in words:
      score += tf[word,sentence]*(math.log(sent_count/sf[word]))
    scores.append(score)
  return scores

In [ ]:
from nltk.tokenize import sent_tokenize
def position_score(text):
  text = sent_tokenize(text)
  sentence_count = len(text)
  sent_score = []
  for i in range(1,len(text)+1):
    pos_score = i/(sentence_count)
    if pos_score > 0.0 and pos_score <= 0.1:
      sent_score.append(0.37/sentence_count)#17
    elif pos_score > 0.1 and pos_score <= 0.2:
      sent_score.append(0.23/sentence_count)
    elif pos_score > 0.2 and pos_score <= 0.3:
      sent_score.append(0.14/sentence_count)
    elif pos_score > 0.3 and pos_score <= 0.4:
      sent_score.append(0.08/sentence_count)
    elif pos_score > 0.4 and pos_score <= 0.5:
      sent_score.append(0.05/sentence_count)
    elif pos_score > 0.5 and pos_score <= 0.6:
      sent_score.append(0.04/sentence_count)
    elif pos_score > 0.6 and pos_score <= 0.7:
      sent_score.append(0.06/sentence_count)
    elif pos_score > 0.7 and pos_score <= 0.8:
      sent_score.append(0.04/sentence_count)
    elif pos_score > 0.8 and pos_score <= 0.9:
      sent_score.append(0.04/sentence_count)
    elif pos_score > 0.9 and pos_score <= 1.0:
      sent_score.append(0.15/sentence_count)
  return sent_score

In [ ]:
from nltk import word_tokenize, pos_tag, ne_chunk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
def pos_tags(text):
  return pos_tag(word_tokenize(text))
def named_entity(text):
  text = pos_tags(text)
  return ne_chunk(text)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
from collections import Counter
def noun_counter(text):
  counts = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    a = pos_tags(sentence)
    count = Counter(tag for word,tag in a)
    counts.append(count['NN'])
  if len(counts)>0:
    m = max(counts)
  else:
    m = 1
  counts = [(count/m) for count in counts]
  return counts

In [ ]:
def sent_length_score(text):
  text = sent_tokenize(text)
  max_len = max(len(i) for i in text)
  sent_lengths = []
  for sentence in text:
    sent_lengths.append(len(sentence)/max_len)
  return sent_lengths

In [ ]:
def sent_numerical_score(text):
  num_scores = []
  text = sent_tokenize(text)
  for sentence in text:
    count = 0
    words = word_tokenize(sentence)
    sent_len = len(words)
    for word in words:
      if word.replace('.','',1).isdigit():
        count+=1
    num_scores.append(count/sent_len)
  return num_scores

In [ ]:
def sentiment_score(text):
  from textblob import TextBlob
  sentences = sent_tokenize(text)
  scores = []
  for sentence in sentences:
    blob = TextBlob(text)
    score = blob.sentiment.polarity
    if(score<0):
      score *=-1
    scores.append(score)
  return scores

Pre-Processing

In [ ]:
def text_cleaner(text):
  new_text = text.lower()
  new_text = re.sub("[^a-zA-Z0-9 .?]", " ", str(text))    # removing anything other than english letters, digits, full stops and question marks
  new_text = re.sub("[ +]", " ", new_text) # removing unwanted whitespaces
  new_text = re.sub("[.]{2,}","", new_text) # removing unwanted period symbols
  return new_text

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def stopwords_remover(text):
  word_tokens = word_tokenize(text)
  new_text = ""
  stop_words = set(stopwords.words('english'))
  for words in word_tokens:
    if words not in stop_words:
      new_text = new_text + words + " "
  return new_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('wordnet')
def stemmer(text):
  from nltk.stem import PorterStemmer
  ps =PorterStemmer()
  sentences = sent_tokenize(text)
  new_text = ""
  for sentence in sentences:
    temp_text = ""
    words = word_tokenize(sentence)
    for word in words:
      word = ps.stem(word)
      temp_text = temp_text + word + " "
    new_text = new_text + temp_text
  return new_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def new_model(text,output_len):

  import warnings
  warnings.filterwarnings("ignore")

  sentences = nltk.sent_tokenize(text)
  #print("No of sentences in input :",len(sentences))

  if(output_len==None):
    output_len = len(sentences)//5
  if(len(sentences)<output_len):
    print("Output length is greater than input length")
    return
  
  #text = text_cleaner(text)
  scores = []
  s = nltk.sent_tokenize(text)
  score1 = tf_isf_score(text)
  score2 = position_score(text)
  score3 = sent_length_score(text)
  score4 = sent_numerical_score(text)
  score5 = noun_counter(text)
  score6 = sentiment_score(text)
  scores_2 = []
  for i in range(len(s)):
    scores_2.append(score1[i]+score2[i]+score3[i]+score4[i]+score5[i]+score6[i])
  
  topics = topic_model(text)
  nps = noun_phrases(text)
  if(len(nltk.sent_tokenize(text))!=len(nps)):
    print("Wrong tokenization")
    return
  scores_1 = []
  for np_sent in nps:
    score = []
    for np_each in np_sent:
      score_each_np = 1
      np_words = nltk.word_tokenize(np_each)
      for np_each_word in np_words:
        score_each_np_word = 0
        for topic in topics:
          score_each_np_word += find_distance(topic,np_each_word)
        score_each_np *= score_each_np_word
      score.append(score_each_np)
    scores_1.append(sum(score))

  
  if(len(scores_1)!=len(scores_2)):
    print("Diff score lengths",len(scores_1),len(scores_2))
    return
  
  for i in range(len(scores_1)):
    scores.append(scores_1[i]+scores_2[i])
  data_op = pd.DataFrame({'text':sentences,'scores':scores})
  data_op = data_op.sort_values(by=['scores'],ascending=False)

  x = data_op.values
  new_text = ""
  for i in range(output_len):
    new_text += x[i][0]+" "
  return new_text

In [ ]:
text = "A car (or automobile) is a wheeled motor vehicle used for transportation. Most definitions of cars say that they run primarily on roads, seat one to eight people, have four tires, and mainly transport people rather than goods.[2][3]Cars came into global use during the 20th century, and developed economies depend on them. The year 1886 is regarded as the birth year of the modern car when German inventor Karl Benz patented his Benz Patent-Motorwagen. Cars became widely available in the early 20th century. One of the first cars accessible to the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced animal-drawn carriages and carts, but took much longer to be accepted in Western Europe and other parts of the world.[citation needed]Cars have controls for driving, parking, passenger comfort, and a variety of lights. Over the decades, additional features and controls have been added to vehicles, making them progressively more complex, but also more reliable and easier to operate.[citation needed] These include rear reversing cameras, air conditioning, navigation systems, and in-car entertainment. Most cars in use in the 2010s are propelled by an internal combustion engine, fueled by the combustion of fossil fuels. Electric cars, which were invented early in the history of the car, became commercially available in the 2000s and are predicted to cost less to buy than gasoline cars before 2025.[4][5]There are costs and benefits to car use. The costs to the individual include acquiring the vehicle, interest payments (if the car is financed), repairs and maintenance, fuel, depreciation, driving time, parking fees, taxes, and insurance.[6] The costs to society include maintaining roads, land use, road congestion, air pollution, public health, health care, and disposing of the vehicle at the end of its life. Traffic collisions are the largest cause of injury-related deaths worldwide.[7]The personal benefits include on-demand transportation, mobility, independence, and convenience.[8] The societal benefits include economic benefits, such as job and wealth creation from the automotive industry, transportation provision, societal well-being from leisure and travel opportunities, and revenue generation from the taxes. People's ability to move flexibly from place to place has far-reaching implications for the nature of societies.[9] There are around 1 billion cars in use worldwide. The numbers are increasing rapidly, especially in China, India and other newly industrialized countries[10]."
output_sentence = None
summary = new_model(text,output_sentence)
print("No of sentences in output",len(nltk.sent_tokenize(summary)))
print(summary)

No of sentences in output 4
[8] The societal benefits include economic benefits, such as job and wealth creation from the automotive industry, transportation provision, societal well-being from leisure and travel opportunities, and revenue generation from the taxes. Electric cars, which were invented early in the history of the car, became commercially available in the 2000s and are predicted to cost less to buy than gasoline cars before 2025. Cars were rapidly adopted in the US, where they replaced animal-drawn carriages and carts, but took much longer to be accepted in Western Europe and other parts of the world. Most definitions of cars say that they run primarily on roads, seat one to eight people, have four tires, and mainly transport people rather than goods. 


In [ ]:
input_text = input("Enter the text to be summarised :")
output_sentence = None
summary = new_model(input_text,output_sentence)
print("No of sentences in output :",len(nltk.sent_tokenize(summary)))
print(summary)

Enter the text to be summarised :A car (or automobile) is a wheeled motor vehicle used for transportation. Most definitions of cars say that they run primarily on roads, seat one to eight people, have four tires, and mainly transport people rather than goods.[2][3]Cars came into global use during the 20th century, and developed economies depend on them. The year 1886 is regarded as the birth year of the modern car when German inventor Karl Benz patented his Benz Patent-Motorwagen. Cars became widely available in the early 20th century. One of the first cars accessible to the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced animal-drawn carriages and carts, but took much longer to be accepted in Western Europe and other parts of the world.[citation needed]Cars have controls for driving, parking, passenger comfort, and a variety of lights. Over the decades, additional features and controls have b